The big thing to try here is to break down the problem into two different types of problems:
- when the number is even (e.g. t(6))
- when the number is odd (e.g. t(7))

Since there must a palindrome the same elements must exist on both the left and right sides. Since they are the same elements, it is essentially saying 2 * {elements}, which will always result in an even numbered sum and even number of elements.

If we want to have an odd 'n', then we will need to have an odd number of elements and the middle number *MUST* be odd. 

If we want to have an even 'n', then we can either have (1) an even number of elements where both sides are the same or (2) an odd number of elements where the middle element is an even number.

Rather than trying to generate all of the compositions for a number 'n', we can take the idea that it is a palindrome and the idea that it must be a mirror and get the total number of compositions for numbers that are larger than the sum of the composition!

How do we get the compositions?

We know how to calculate the total number of partitions, but how many different ways are there when order matters?

In [14]:
import math
import itertools
import time

In [2]:
def partitions(n):
    # base case of recursion: zero is the sum of the empty list
    if n == 0:
        yield []
        return

    # modify partitions of n-1 to form partitions of n
    for p in partitions(n-1):
        yield [1] + p
        if p and (len(p) < 2 or p[1] > p[0]):
            yield [p[0] + 1] + p[1:]

In [3]:
def num_compositions_longhand(num):
    
    """
    Get the number of compositions for a number, num
    
    This is the ways that an interger can be written as the sum of a sequence of positive integers
    Order matters.
    """
    
    # generate all of the partitions
    parts = [tuple(x) for x in partitions(num)]
    
    comps = set()
    for part in parts:
        perms = itertools.permutations(part)
        for perm in perms:
            comps.add(perm)
            
#     print(comps)
    
    return len(comps)

In [4]:
def num_compositions(num):
    return 2 ** (num - 1)

In [5]:
for i in range(1, 12 + 1):
    print(f"{i}. {num_compositions(i)} vs. 2^{i-1} == {2**(i-1)}")

1. 1 vs. 2^0 == 1
2. 2 vs. 2^1 == 2
3. 4 vs. 2^2 == 4
4. 8 vs. 2^3 == 8
5. 16 vs. 2^4 == 16
6. 32 vs. 2^5 == 32
7. 64 vs. 2^6 == 64
8. 128 vs. 2^7 == 128
9. 256 vs. 2^8 == 256
10. 512 vs. 2^9 == 512
11. 1024 vs. 2^10 == 1024
12. 2048 vs. 2^11 == 2048


Interestingly, there are 2^(n-1) compositions for any number, n


How many compositions contain a 2?

let F(n) be the number of compositions of the number 'n' that contain a 2.

We know that

F(0) = 0

F(1) = 0

F(2) = 1 {(2)}

F(3) = 2 {(1, 2), (2, 1)}

F(4) = 4 {(1, 1, 2), (1, 2, 1), (2, 1, 1), (2, 2)}

F(5) = 9 {(1, 1, 1, 2), (1, 1, 2, 1), (1, 2, 1, 1), (2, 1, 1, 1), (1, 2, 2), (2, 1, 2), (2, 2, 1), (2, 3), (3, 2)}

We need to determine the number of compositions with a 2 in them

In [6]:
def compositions_with_2_longhand(num):
    
    # generate all of the partitions
    parts = [tuple(x) for x in partitions(num) if 2 in x]
    
    comps = set()
    for part in parts:
        perms = itertools.permutations(part)
        for perm in perms:
            comps.add(perm)
            
    return comps

In [13]:
print("Total compositions with a '2':")

for i in range(12+1):
    print(f"{i}. {len(compositions_with_2_longhand(i))}")

Total compositions with a '2':
0. 0
1. 0
2. 1
3. 2
4. 4
5. 9
6. 20
7. 43
8. 91
9. 191
10. 398
11. 824
12. 1697


This is going to be way too slow, and so let's look at some of these compositions to be able to determine how to achieve it faster.

How many compositions don't contain a 2?

In [8]:
for i in range(12+1):
    print(f"{i}. Comps w/o a 2: {2**(i-1)- len(compositions_with_2_longhand(i))}")

0. Comps w/o a 2: 0.5
1. Comps w/o a 2: 1
2. Comps w/o a 2: 1
3. Comps w/o a 2: 2
4. Comps w/o a 2: 4
5. Comps w/o a 2: 7
6. Comps w/o a 2: 12
7. Comps w/o a 2: 21
8. Comps w/o a 2: 37
9. Comps w/o a 2: 65
10. Comps w/o a 2: 114
11. Comps w/o a 2: 200
12. Comps w/o a 2: 351


In [9]:
c_no_2 = [0, 1, 1]

for i in range(3, 12+1):
    c_no_2.append(2**(i-1) - len(compositions_with_2_longhand(i)))
    
c_no_2

[0, 1, 1, 2, 4, 7, 12, 21, 37, 65, 114, 200, 351]

In [10]:
prev_sum = [0, 1]

for i in range(2, len(c_no_2) + 1):
    s = 0
    for j in range(i):
        s += c_no_2[j]
    prev_sum.append(s)
    
prev_sum

[0, 1, 1, 2, 4, 8, 15, 27, 48, 85, 150, 264, 464, 815]

So it's relatively obvious that n < 2 means that there will be 0 2's in the composition. 

We also know that there will be fewer than 2^(n-1) compositions that contain a 2, because 2^(n-1) is the max number of compositions for any n.

If we know the total number of compositions for any number n-2, then we can simply add a 2 to each of those compositions and get a composition that sums to n and contains a 2.

num_compositions(n) = 2^(n-1)

num_compositions(n-2) = 2^((n-2) - 1)

thus,

num_compositions(n-2) = 2^(n-3)

NOTE: this only applies when n >= 3

How many compositions that contain '2' exist with other digits?



For each number, k, less than our target number, n. We would have already calculated the total number of compositions that contain a 2. That is given by F(k).

Since they already contain a 2, we can add a number, j = n - k, to the front to let it become a composition for n that contains a 2.

E.g. Let n = 5

Our final answer should be 9.

We know that there are 2^(n-3) compositions that just need a 2 appended to them:

All of the compositions for n-2 (== 3): 
* (1, 1, 1) 
* (1, 2)
* (2, 1)
* (3)


If we look at each of values for F(k) where k < n:
* k = 2
    * (2)
* k = 3
    * (1, 2)
    * (2, 1)
* k = 4
    * (1, 1, 2)
    * (1, 2, 1)
    * (2, 1, 1)
    * (2, 2)

So, if we add the requisite number to these compositions, we should get the full set of compositions:

Let n = 5

* compositions from n-2 with a 2 in front:
    * (2, 1, 1, 1)
    * (2, 1, 2)
    * (2, 2, 1)
    * (2, 3)
* compositions from k (< n) and adding n-k to the front
    * k = 2
        * (3, 2)
    * k = 3
        * (2, 1, 2)
        * (2, 2, 1)
    * k = 4
        * (1, 1, 1, 2)
        * (1, 1, 2, 1)
        * (1, 2, 1, 1)
        * (1, 2, 2)

Notice that k = 3 duplicates results from n-2, and so we should eliminate F(3) to give us the 9 total compositions containing a 2:

* (2, 1, 1, 1)
* (2, 1, 2)
* (2, 2, 1)
* (2, 3)
* (3, 2)
* (1, 1, 1, 2)
* (1, 1, 2, 1)
* (1, 2, 1, 1)
* (1, 2, 2)

Let's try for n = 6:

* compositions from n = 4 w/ a 2 in front:
    * (2, 1, 1, 1, 1)
    * (2, 1, 1, 2)
    * (2, 1, 2, 1)
    * (2, 2, 1, 1)
    * (2, 1, 3)
    * (2, 3, 1)
    * (2, 2, 2)
    * (2, 4)
    
* compositions from F(k) when k < n w/ a k in front:
    * k = 2
        * (4, 2)
    * k = 3
        * (3, 1, 2)
        * (3, 2, 1)
    * k = 4
        * (2, 1, 1, 2)
        * (2, 1, 2, 1)
        * (2, 2, 1, 1)
        * (2, 2, 2)
    * k = 5
        * (1, 2, 1, 1, 1)
        * (1, 2, 1, 2)
        * (1, 2, 2, 1)
        * (1, 2, 3)
        * (1, 3, 2)
        * (1, 1, 1, 1, 2)
        * (1, 1, 1, 2, 1)
        * (1, 1, 2, 1, 1)
        * (1, 1, 2, 2)
        
Again, we will have to eliminate the case when k = n-2 because those are already covered in the earlier compositions.

Let's code it up!

Note we don't actually have to write out the compositions. We know that it is simply:

F(n) = sum of:
* total compositions for n - 2
* sum of compositions of F(k) for k < n
* -(F(n-2))

In [23]:
t0 = time.time()

# initialize values for F(n)
F = [0, 0, 1]

# choose a number that we can see how quickly it can calculate relatively large numbers
for n in range(3, 40):
    
    # get the total compositions for n-2
    F_n = 2**(n-3)
    
    # add the sum of all values for k when k < n
    F_n += sum(F)
    
    # remove duplicates by subtracting F(n-2)
    F_n -= F[-2]
    
    F.append(F_n)
    
for i in F:
    print(i)
    
print()
print(f"{time.time() - t0} sec")

0
0
1
2
4
9
20
43
91
191
398
824
1697
3480
7111
14487
29439
59694
120820
244153
492716
993171
1999923
4023679
8089182
16251760
32632321
65490672
131377999
263452079
528125695
1058395038
2120551916
4247705401
8506995748
17034321659
34104320267
68271249215
136652369006
273497547432

0.0020334720611572266 sec


So now if we want a number, we can quite easily generate the number of sides that would form a palindrome.

We just have to construct palindromes now.

Let x be a value for which the palindrome sums in our equation for t(2n):


If 2n is even, then there can be either
* an odd number of elements in its set, with a center element that is even
* an even number of elements in its set (thus no center)

If 2n is odd, then there must be
* an odd number of elements with a center element that is odd

Let's first focus on the evens.

If there are an even number of elements in the side of the palindrome, then there are F(n) choices to create the palindrome.

e.g. if we are looking for t(2x) = t(6), we look to x = 3

F(3) = 2

* (1, 2)
* (2, 1)

So there will then be 2x twopals that have even quantities of elements:

* (1, 2) + (2, 1) = (1, 2, 2, 1)
* (2, 1) + (1, 2) = (2, 1, 1, 2)

$\therefore\$ F(x) where x = n/2

If there are an odd number of elements, then the situation is slightly more complicated.

if we call one side of the palindrome set, p, with number of elements in it 'q'. Then the following must be true:

* the total number of elements in the twopal will be 2p + 1
* the center element must be even
    * 2p will always be even, and since we have only 1 additional element to add, it must also be even to end in an even number

Since 2 is a special case here, we should consider '2' different than other even numbers.

If '2' is a center, then it already is a twopal and we don't care what the numbers are.

So, given that our target number is some value 'n', and the center is 2, then 2*sum(p) = n - 2

So we can check F(k) when k = x - 1 (x = n/2)

For any k, there are 2^(k-1) total compositions for that number, and so if k = x - 1

Then there are 2^((x-1) - 1) = 2^(x-2) possible twopals where 2 is the middle element.

$\therefore\$ 2^(x-2) where x = n/2

In other situations, the center will have an even value between 4 and n.

We now care whether the set p has a '2' in it because we are not guaranteed to have one in the full set otherwise.

Thus, we should be using values for F(k).

We need to start at 4, and we need to go until x, using only even values

$\therefore\$

\begin{equation}
    \sum^{x}_{j=2} F(2*j)
\end{equation}

Adding all of those together, we should get the final summation.

if n is even, then
\begin{equation}
    t(n) = 2^{\frac{n}{2}-2} + f(n/2) + \sum^{n/2}_{k=4}f(k)
\end{equation}

Let's try it for n = 6:

t(6) = 2^(n/2 - 2) + f(n/2) + sum^{n/2}_{j=2}(f(2*j))

t(6) = 2^(3-2) + f(3) + sum()

t(6) = 2 + 2 = 4

Now let's deal with odds...

There must be a middle element, and that element must be odd.

Thus the sides must have a '2' in it. So we should use f(k).

if we let n = 2*k + j, then for any value of j <= n, there must be f(k) possible twopals.

$\therefore\$

\begin{equation}
    \sum^{\frac{n-1}{2}}_{k=0} f(k)
\end{equation}

In [5]:
f = [0, 0, 1]

k = 1

# deal with evens
n = 2*k

t_n = 2 ** (n-2)
t_n -= f[n-1]
sub_sum = 0
for k in range(n+1):
    sub_sum += f[n]
t_n += sub_sum

4